# Testing integration with `vegas`

__Author:__ A. J. Tropiano [atropiano@anl.gov]<br/>
__Date:__ October 11, 2022

Trying to do a spectroscopic overlap calculation evaluating the integrals in the $\delta U^\dagger$ term with Monte Carlo integration using `vegas`. Currently attempting to do all the sums in the expression by looping (excluding as many terms equal to zero as possible), and use `vegas` only for integration over momenta.

_Last update:_ April 5, 2023

### Install `vegas`

In [1]:
# %%bash
# pip install vegas

In [2]:
# Python imports
import functools
# from numba import njit, vectorize, guvectorize, float64
import numpy as np
import numpy.linalg as la
import pandas as pd
from scipy.interpolate import interp1d, RectBivariateSpline
from scipy.special import spherical_jn, sph_harm
import shutil
from sympy.physics.quantum.cg import CG
import time
import vegas

In [3]:
# Imports from scripts
# ...

## Example from tutorial [online](https://vegas.readthedocs.io/en/latest/tutorial.html)

In [4]:
def f(x):
    dx2 = 0
    for d in range(4):
        dx2 += (x[d] - 0.5) ** 2
    return np.exp(-dx2 * 100.) * 1013.2118364296088

In [5]:
integ = vegas.Integrator([[-1, 1], [0, 1], [0, 1], [0, 1]])
integ(f, nitn=10, neval=1e3)
%time result = integ(f, nitn=10, neval=1e4)
print(result.summary())
print('result = %s    Q = %.2f' % (result, result.Q))

CPU times: user 181 ms, sys: 607 µs, total: 182 ms
Wall time: 182 ms
itn   integral        wgt average     chi2/dof        Q
-------------------------------------------------------
  1   0.9947(33)      0.9947(33)          0.00     1.00
  2   1.0044(24)      1.0011(19)          5.61     0.02
  3   1.0015(20)      1.0013(14)          2.82     0.06
  4   0.9993(18)      1.0005(11)          2.14     0.09
  5   0.9986(16)      0.99994(91)         1.83     0.12
  6   1.0004(14)      1.00007(77)         1.48     0.19
  7   0.9990(13)      0.99980(66)         1.31     0.25
  8   0.9996(13)      0.99976(59)         1.13     0.34
  9   0.9995(12)      0.99972(53)         0.99     0.44
 10   0.9997(12)      0.99971(49)         0.88     0.54

result = 0.99971(49)    Q = 0.54


In [6]:
result.sdev

0.00048742789438680016

## Integrand depends on other parameters

Here we use `functools.partial` to input parameters into the integrand function.
This will be useful for practical purposes where the integrand depends on things other than integration variables.

In [7]:
def g(y, x):
    dx2 = 0
    for d in range(4):
        dx2 += (x[d] - y) ** 2
    return np.exp(-dx2 * 100.) * 1013.2118364296088

In [8]:
# Results here should be the same!
x = np.array((0.1, 0.2, 0.3, 0.1))
print(f(x), g(0.5, x))

2.900337707812307e-17 2.900337707812307e-17


In [9]:
integ = vegas.Integrator([[-1, 1], [0, 1], [0, 1], [0, 1]])
y = 0.5
integrand = functools.partial(g, y)
integ(integrand, nitn=10, neval=1e3)
%time result = integ(integrand, nitn=10, neval=1e4)
print(result.summary())
print('result = %s    Q = %.2f' % (result, result.Q))

CPU times: user 184 ms, sys: 1.31 ms, total: 185 ms
Wall time: 185 ms
itn   integral        wgt average     chi2/dof        Q
-------------------------------------------------------
  1   0.9988(32)      0.9988(32)          0.00     1.00
  2   1.0050(24)      1.0028(19)          2.29     0.13
  3   0.9976(20)      1.0003(14)          2.86     0.06
  4   0.9996(18)      1.0000(11)          1.94     0.12
  5   1.0006(16)      1.00021(90)         1.47     0.21
  6   1.0011(14)      1.00048(76)         1.24     0.29
  7   0.9988(14)      1.00011(67)         1.22     0.29
  8   1.0000(12)      1.00008(59)         1.05     0.40
  9   1.0002(11)      1.00011(52)         0.92     0.50
 10   0.9990(12)      0.99992(48)         0.91     0.52

result = 0.99992(48)    Q = 0.52


## Batch integrand

Function `f_batch(x)` accepts an array of integration points `x[i, d]` where `i=0...` labels the integration point and `d=0...` the direction and returns an array of integrand values corresponding to those points.
The decorator `vegas.batchintegrand()` tells `vegas` that it should send integration points to `f(x)` in batches.

In [10]:
def f(x):
    """Example integrand which returns a float given an integration point x with
    shape (4,).
    """
    dim = len(x)
    norm = 1013.2118364296088 ** (dim / 4.)
    dx2 = 0.0
    for d in range(dim):
        dx2 += (x[d] - 0.5) ** 2
    return np.exp(-100. * dx2) * norm

# integ = vegas.Integrator(4 * [[0, 1]])
integ = vegas.Integrator([[0,1],[0,1],[0,1],[0,1]])

integ(f, nitn=10, neval=2e5)
%time result = integ(f, nitn=10, neval=2e5)
print(result.summary())
print('result = %s    Q = %.2f' % (result, result.Q))

CPU times: user 3.66 s, sys: 15.4 ms, total: 3.68 s
Wall time: 3.68 s
itn   integral        wgt average     chi2/dof        Q
-------------------------------------------------------
  1   0.99947(36)     0.99947(36)         0.00     1.00
  2   0.99991(32)     0.99971(24)         0.82     0.37
  3   0.99975(29)     0.99973(19)         0.41     0.66
  4   0.99968(27)     0.99971(15)         0.28     0.84
  5   1.00008(24)     0.99982(13)         0.61     0.66
  6   1.00005(22)     0.99988(11)         0.65     0.66
  7   0.99995(21)     0.999892(99)        0.56     0.76
  8   0.99968(20)     0.999851(89)        0.61     0.75
  9   0.99999(20)     0.999875(81)        0.59     0.79
 10   1.00009(19)     0.999907(75)        0.64     0.76

result = 0.999907(75)    Q = 0.76


In [11]:
@vegas.batchintegrand
def f_batch(x):
    """Example integrand which evaluates multiple points simultaneously. Here x
    is several integration points with shape (N,4), where N is the number of
    integration points and 4 is the dimension of the integral (meaning there are
    four integration varibles).
    """
    # evaluate integrand at multiple points simultaneously
    dim = x.shape[1]
    norm = 1013.2118364296088 ** (dim / 4.)
    dx2 = 0.0
    for d in range(dim):
        dx2 += (x[:, d] - 0.5) ** 2
    return np.exp(-100. * dx2) * norm

integ = vegas.Integrator(4 * [[0, 1]])

integ(f_batch, nitn=10, neval=2e5)
%time result = integ(f_batch, nitn=10, neval=2e5)
print(result.summary())
print('result = %s    Q = %.2f' % (result, result.Q))

CPU times: user 158 ms, sys: 5.6 ms, total: 163 ms
Wall time: 163 ms
itn   integral        wgt average     chi2/dof        Q
-------------------------------------------------------
  1   1.00067(36)     1.00067(36)         0.00     1.00
  2   1.00014(32)     1.00038(24)         1.19     0.27
  3   0.99980(29)     1.00015(19)         1.75     0.17
  4   0.99979(27)     1.00003(15)         1.59     0.19
  5   0.99996(24)     1.00001(13)         1.21     0.31
  6   0.99995(23)     1.00000(11)         0.98     0.43
  7   0.99999(21)     0.999994(99)        0.81     0.56
  8   1.00006(20)     1.000007(89)        0.71     0.66
  9   1.00001(20)     1.000007(81)        0.62     0.76
 10   1.00006(19)     1.000016(75)        0.56     0.83

result = 1.000016(75)    Q = 0.83


In [12]:
@vegas.batchintegrand
class f_batch_class:
    def __init__(self, dim, y):
        self.dim = dim
        self.norm = 1013.2118364296088 ** (dim / 4.)
        self.y = y

    def __call__(self, x):
        # evaluate integrand at multiple points simultaneously
        dx2 = 0.0
        for d in range(self.dim):
            dx2 += (x[:, d] - self.y) ** 2
        return np.exp(-100. * dx2) * self.norm

f = f_batch_class(dim=4, y=0.5)
integ = vegas.Integrator(f.dim * [[0, 1]])

integ(f, nitn=10, neval=2e5)
%time result = integ(f, nitn=10, neval=2e5)
print(result.summary())
print('result = %s    Q = %.2f' % (result, result.Q))

CPU times: user 157 ms, sys: 3.86 ms, total: 161 ms
Wall time: 161 ms
itn   integral        wgt average     chi2/dof        Q
-------------------------------------------------------
  1   1.00037(35)     1.00037(35)         0.00     1.00
  2   0.99993(32)     1.00013(24)         0.83     0.36
  3   1.00013(29)     1.00013(18)         0.42     0.66
  4   1.00030(27)     1.00019(15)         0.36     0.78
  5   0.99926(24)     0.99993(13)         2.89     0.02
  6   1.00012(23)     0.99997(11)         2.42     0.03
  7   1.00011(21)     1.000005(99)        2.08     0.05
  8   1.00030(20)     1.000062(89)        2.03     0.05
  9   0.99993(20)     1.000040(81)        1.82     0.07
 10   1.00004(20)     1.000040(75)        1.62     0.10

result = 1.000040(75)    Q = 0.10


## Sums with `vegas`

$$
\sum_{i=0}^{N-1} f(i) = N \int_0^1 dx f(\mathrm{floor}(xN))
$$

In [20]:
def ridge(x):
    """Integrand: ridge of N Gaussians spread evenly along the diagonal"""
    N = 10000
    dim = 4
    x0 = 0.4 + 0.2 * np.floor(x[-1] * N) / (N - 1.)
    dx2 = 0.0
    for xd in x[:-1]:
        dx2 += (xd - x0) ** 2
    return np.exp(-100. * dx2) *  (100. / np.pi) ** (dim / 2.)

In [21]:
def main():
    integ = vegas.Integrator(5 * [[0, 1]])
    # adapt
    integ(ridge, nitn=10, neval=5e4)
    # final results
    result = integ(ridge, nitn=10, neval=5e4)
    print('result = %s    Q = %.2f' % (result, result.Q))

In [22]:
main()

result = 0.99968(83)    Q = 0.36
